## RAG app TFM
In this use case it is shown how to exctract information from a PDF file through LLM queries with RAG (Retrieval Augmented Generation) technology. For this use case is necessary the use of a vector database (in this case FAISS), embeddings and OpenAI model calls. To show the final result, the model is embedded on a Gradio UI.

In [1]:
from dotenv import load_dotenv
import os

load_dotenv("apis.env")
# hf_api_key = os.environ['HF_API_KEY']

True

In [2]:
# PDF to ectract info from
pdf_path = "BOE-A-1978-31229-consolidado.pdf"

# Load pdf with external info not seen during training of the LLM
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader(pdf_path)
pages = loader.load_and_split()

In [3]:
pages[0]

Document(page_content='Constitución Española.\nCortes Generales\n«BOE» núm. 311, de 29 de diciembre de 1978\nReferencia: BOE-A-1978-31229\nÍNDICE\n   \nPreámbulo ................................................................ 3\nTÍTULO PRELIMINAR ........................................................... 3\nTÍTULO I. De los derechos y deberes fundamentales ........................................ 4\nCAPÍTULO PRIMERO. De los españoles y los extranjeros ................................... 5\nCAPÍTULO SEGUNDO. Derechos y libertades .......................................... 5\nSección 1.ª De los derechos fundamentales y de las libertades públicas ........................ 5\nSección 2.ª De los derechos y deberes de los ciudadanos ................................. 8\nCAPÍTULO TERCERO. De los principios rectores de la política social y económica ................... 10\nCAPÍTULO CUARTO. De las garantías de las libertades y derechos fundamentales .................. 12\nCAPÍTULO QUINTO. De la

In [4]:
# Generate vector space representation with words from the external data
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [5]:
# Load embeddings in vector database
from langchain.vectorstores import FAISS
db = FAISS.from_documents(pages, embeddings)

In [50]:
input = "Existe algún modo rápido para triturar alimentos?"
chain(input, return_only_outputs=True)

{'result': ' Sí, se puede utilizar el modo "Turbo" para triturar alimentos'}

# Advanced Gradio app

In [49]:
import gradio as gr

theme = gr.themes.Base(
    primary_hue="rose",
).set(
    body_background_fill='*neutral_50',
    body_text_color='*neutral_500',
    body_text_weight='300',
    background_fill_primary='*neutral_50',
    background_fill_secondary='*primary_50',
    border_color_primary='*primary_400',
    color_accent_soft='*primary_300',
    link_text_color='*primary_300',
    link_text_color_active='*neutral_300',
    link_text_color_hover='*primary_100',
    link_text_color_visited='*neutral_400',
    code_background_fill='*primary_200',
    button_secondary_background_fill='*neutral_100',
    button_secondary_border_color='*neutral_900',
    button_secondary_text_color='*primary_400',
    button_cancel_background_fill='*primary_600',
    button_cancel_background_fill_hover='*primary_700',
    button_cancel_text_color='*neutral_50',
    slider_color='*primary_500'
)


In [55]:
import gradio as gr
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

def extract_sentence(dictionary):
    # Extract the value associated with the key 'result'
    sentence = dictionary.get('result', '')
    return sentence

def model_hyperparameters(temperature, max_tokens):#
    llm = OpenAI(
        model_name="gpt-3.5-turbo-instruct",
        temperature=temperature,
        max_tokens=max_tokens,
        streaming=True
        )
    chain = RetrievalQA.from_llm(llm=llm, retriever=db.as_retriever())
    return chain

def respond(message,temperature=0.7, max_tokens=32):
    prompt = message
    chain = model_hyperparameters(temperature, max_tokens)
    completion = chain(prompt, return_only_outputs=True)
    return extract_sentence(completion)

with gr.Blocks(theme=theme) as demo: 
    gr.Markdown("# TFM RAG App")
    gr.Markdown("With this RAG app, you can generate augmented responses from la Constitución Española.")

    msg = gr.Textbox(label="Ask a question")
    with gr.Accordion(label="Advanced options",open=False):
        temperature = gr.Slider(label="temperature", minimum=0.1, maximum=1.0, value=0.2, step=0.1)
        max_tokens = gr.Slider(label="Max tokens", value=64, maximum=256, minimum=8, step=1)
    completion = gr.Textbox(label="Response")
    btn = gr.Button("Submit", variant="primary")
    clear = gr.ClearButton(components=[msg, completion], value="Clear console", variant="stop")

    btn.click(respond, inputs=[msg, temperature, max_tokens], outputs=[completion])
    msg.submit(respond, inputs=[msg, temperature, max_tokens], outputs=[completion])

gr.close_all()
demo.queue().launch(share=True)    

Running on local URL:  http://127.0.0.1:7889
Running on public URL: https://4c593117ca3b342302.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [32]:
gr.themes.builder()

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----

To create a public link, set `share=True` in `launch()`.


Exception in thread Thread-67 (_do_normal_analytics_request):
Traceback (most recent call last):
  File "c:\Users\Nacho\anaconda3\envs\TFM\Lib\site-packages\httpcore\_exceptions.py", line 10, in map_exceptions
    yield
  File "c:\Users\Nacho\anaconda3\envs\TFM\Lib\site-packages\httpcore\_backends\sync.py", line 136, in write
    n = self._sock.send(buffer)
        ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Nacho\anaconda3\envs\TFM\Lib\ssl.py", line 1243, in send
    return self._sslobj.write(data)
           ^^^^^^^^^^^^^^^^^^^^^^^^
TimeoutError: The write operation timed out

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\Nacho\anaconda3\envs\TFM\Lib\site-packages\httpx\_transports\default.py", line 66, in map_httpcore_exceptions
    yield
  File "c:\Users\Nacho\anaconda3\envs\TFM\Lib\site-packages\httpx\_transports\default.py", line 228, in handle_request
    resp = self._pool.handle_request(req)
           ^^^